In [17]:
import pandas as pd
import numpy as np
import sqlite3 as db
from io import StringIO
from csv import writer 

In [18]:
con = db.connect('accounts.db')

In [19]:
# read all data from decay_amount table
# NOTE: The most important information we need from here is the initial_claimable_amount. 
# Nothing else is used

sql = 'SELECT * FROM decay_amount'
decay_amount = pd.read_sql(sql, con)

# clean empty data
decay_amount['initial_claimable_amount']=decay_amount.initial_claimable_amount.replace('',np.nan).astype(float)
decay_amount['vote_action']=decay_amount.vote_action.replace('',np.nan).astype(float)
decay_amount['ibc_action']=decay_amount.ibc_action.replace('',np.nan).astype(float)
decay_amount['delegate_action']=decay_amount.delegate_action.replace('',np.nan).astype(float)
decay_amount['evm_action']=decay_amount.evm_action.replace('',np.nan).astype(float)
decay_amount['total_lost']=decay_amount.total_lost.replace('',np.nan).astype(float)
decay_amount['total_claimed']=decay_amount.total_claimed.replace('',np.nan).astype(float)

decay_amount.count()

id                          168662
sender                      168662
vote_action                 122772
ibc_action                  106613
delegate_action             120183
evm_action                  132062
total_claimed               168662
total_lost                  155341
initial_claimable_amount    155341
total_lost_evmos            168662
dtype: int64

In [20]:
# read all claims events from claim_event table
# NOTE: This represents all claims events in the chain's history

sql = 'SELECT * FROM claim_event'
claim_event = pd.read_sql(sql, con)

claim_event['amount'] = claim_event['amount'].str.replace('aevmos', '').astype(float)
claim_event['height'] = claim_event['height'].astype(int)

claim_event.count()

id              520043
sender          520043
height          520043
amount          520043
claim_action    520043
dtype: int64

In [21]:
# read all merge events from merged_event table
# NOTE: This represents all merge events in the chain's history

sql = 'SELECT * FROM merged_event'
merge_event = pd.read_sql(sql, con)

merge_event['claimed_coins'] = merge_event['claimed_coins'].str.replace('aevmos', '').astype(float)
merge_event['fund_community_pool_coins'] = merge_event['fund_community_pool_coins'].str.replace('aevmos', '').astype(float)
merge_event['height'] = merge_event['height'].astype(int)
merge_event['sender_genesis_claim_record'] = merge_event['sender_genesis_claim_record'].astype(float)

merge_event.count()

id                             1022
recipient                      1022
sender                         1022
height                         1022
claimed_coins                  1022
fund_community_pool_coins      1022
sender_evmos_prefix            1022
sender_genesis_claim_record     950
dtype: int64

# Affected Accounts Calculation
The goal is to generate a table from our `claims` and `merged_events` events collection in which we can calculate the amount lost by address due to the decay bug.

The generated table will have the following columns:
- `sender` - the address that received the claim rewards
- `height` - the height at which the event occurred
- `event_type` - action type of the event that got triggered
    - ACTION_EVM
    - ACTION_DELEGATE
    - ACTION_VOTE
    - ACTION_IBC_TRANSFER
- `initial_claimable_amount` - claim amount available at genesis for the `sender` if any
- `claimable_at_event` - the amount available to claim for the `sender` at the event if any.
    - If there were any `merge_claims_records`  events present for the `sender` account before the event `height`, then the claimable amount at genesis of both accounts were taking into account and added.
- `expected_claim` - `claimable_at_event` / 4
- `amount_claimed` - the amount the address actually received on the event
- `lost_amount` - the difference between `claimable_at_event` and `amount_claimed`
    - This is the amount that should be returned.

Firt we will callect this data for the `claims` event and then do the same for the `merged_events`. Lastly we will append the data into a unified database to calculate the final amount to be returned to the users on the Evmos V12 upgrade.
  
Once the data is collected, a `csv` file is generated.

### Claim events affected by decay

In [22]:
# Claim Events Decay loss
# Iterate over all claim events:
# 1. Find the sender in the decay_amount table
# 2. If the sender was found in the decay_amount table, check if the initial_claimable_amount is an integer and add that amount to the claimable_at_event
# 2. Find the sender in the merged_event table
# 3. If the sender was found in the merged_event table, check if the height of the merged_event is less than the height of the claim_event
# 4. If the height of the merged_event is less than the height of the claim_event, then we add the `sender_genesis_claim_record` of the merged event to the claimable_at_event amount
# 5. Calculate the lost amount
# NOTE: This process takes a long time to run as it iterates over more than 500k accounts and its probably not the most efficient way to do it.

count_progress = 0
output = []
for index, row in claim_event.iterrows():
    claimable_at_event = 0
    if count_progress == 1000:
        print('processed ', index)
        count_progress = 0
    # find in dataframe row with COLUMN = VALUE
    decay_row = decay_amount.loc[decay_amount['sender'] == row['sender']]
    initial_claimable_amount = decay_row['initial_claimable_amount'].values[0]
    if initial_claimable_amount.is_integer():
        claimable_at_event = initial_claimable_amount
    # find sender in merged_event and check if was not found
    merge_row = merge_event.loc[merge_event['recipient'] == row['sender']]
    # check if we should increase the claimable amount at that block height
    if not merge_row.empty and merge_row['height'].values[0] < row['height']:
        # increasae claimable_at_event
        claimable_at_event = claimable_at_event + merge_row['sender_genesis_claim_record'].values[0]

    # calculate lost amount
    amount_to_claimed = claimable_at_event / 4
    lost_amount = amount_to_claimed - row['amount']

    output.append((row['id'], row['sender'], row['claim_action'], row['height'], initial_claimable_amount, claimable_at_event, amount_to_claimed, row['amount'], lost_amount))
    count_progress = count_progress + 1
result = pd.DataFrame(output, columns=['id', 'sender','event_type', 'height', 'initial_claimable_amount', 'claimable_at_event', 'expected_claim', 'amount_claimed', 'lost_amount'])
result.head()

processed  1000
processed  2000
processed  3000
processed  4000
processed  5000
processed  6000
processed  7000
processed  8000
processed  9000
processed  10000
processed  11000
processed  12000
processed  13000
processed  14000
processed  15000
processed  16000
processed  17000
processed  18000
processed  19000
processed  20000
processed  21000
processed  22000
processed  23000
processed  24000
processed  25000
processed  26000
processed  27000
processed  28000
processed  29000
processed  30000
processed  31000
processed  32000
processed  33000
processed  34000
processed  35000
processed  36000
processed  37000
processed  38000
processed  39000
processed  40000
processed  41000
processed  42000
processed  43000
processed  44000
processed  45000
processed  46000
processed  47000
processed  48000
processed  49000
processed  50000
processed  51000
processed  52000
processed  53000
processed  54000
processed  55000
processed  56000
processed  57000
processed  58000
processed  59000
proces

,id,sender,event_type,height,initial_claimable_amount,claimable_at_event,expected_claim,amount_claimed,lost_amount
0,1,evmos18xszt8l8slmm873l2v5w20v5tsh4l7en4d0jex,ACTION_DELEGATE,87701,9.323842e+19,9.323842e+19,2.330960e+19,2.330960e+19,0.0
1,2,evmos1v80jrjdzrc5zt3q5c8zwkawm0kqx48z84yq0kc,ACTION_VOTE,87701,7.050327e+18,7.050327e+18,1.762582e+18,1.762582e+18,0.0
2,3,evmos1p3a4h405w5vh4t0q64jsyvm4k8tyx9stujj75x,ACTION_DELEGATE,87701,2.353460e+19,2.353460e+19,5.883650e+18,5.883650e+18,0.0
3,4,evmos1rgf75fc9chvc2x8p4yaad5llxqzjhfd378z0zt,ACTION_VOTE,87701,4.009982e+19,4.009982e+19,1.002496e+19,1.002496e+19,0.0
4,5,evmos1az3xkmtpwdhjuwpg8llm2xh7p0ea8f4pjq4v4r,ACTION_DELEGATE,87701,2.170491e+19,2.170491e+19,5.426226e+18,5.426226e+18,0.0


In [23]:
# Address whose decay loss is bigger than 0
impacted_claim_events_results = result.loc[result['lost_amount'] > 0]

In [ ]:
# generate csv out of impacted_claim_events_results
impacted_claim_events_results.sort_values(by=['sender']).to_csv('claims_decay_loss_results.csv', index=False)

### Merged events affected by decay

Merged events are claim events themselves that are not present within the `claims` table given that they do not emit that event. Because of that this events could also be affected by decay losses. To calculate that we took the `claims_records` on genesis from the sender of the merged event and compare that to what they received for that event. Once we had that data, a table with the same model as the one that was did for the claims results was generated so they can easily be appended.

In [24]:
# generate a new copy of merge_event
merge_event_copy = merge_event.copy()

In [25]:
# create a `lost_amount` column for each event compared to what they should have gotten according to genesis
merge_event_copy['lost_amount'] = merge_event_copy['id'].apply(lambda x: (merge_event_copy.loc[merge_event_copy['id'] == x]['sender_genesis_claim_record'].values[0] / 4) - merge_event_copy.loc[merge_event_copy['id'] == x]['claimed_coins'].values[0])
merge_event_copy.head()

,id,recipient,sender,height,claimed_coins,fund_community_pool_coins,sender_evmos_prefix,sender_genesis_claim_record,lost_amount
0,1,evmos16lgzlre83u3sh6mdu6yc5hvyjt8eejcgq43lqc,osmo14h3yn7h4lezzmtqwn7usuzmd6snnugpsehq90c,273246,1.462502e+20,4.061130e+19,evmos14h3yn7h4lezzmtqwn7usuzmd6snnugpsndzmrz,1.835300e+20,-1.003677e+20
1,2,evmos199g7q4utyss49tkn88pz5c0093gkkw76x5znun,osmo1ermc8wk0yqjwfl47xdcn429hz9zm04v04f2den,273249,2.835015e+20,7.872541e+19,evmos1ermc8wk0yqjwfl47xdcn429hz9zm04v0lngn4f,3.514016e+20,-1.956511e+20
2,3,evmos13mkynsrry2kcrqw28wat8zv4q7thhwwcymukfm,osmo1rckjyht7dl0uhm9qk8k6f2llk0mtz5cyzhq5df,273353,3.101760e+20,8.619350e+19,evmos1rckjyht7dl0uhm9qk8k6f2llk0mtz5cygdz2pn,5.192732e+20,-1.803577e+20
3,4,evmos16xz0swftu499wg2aauum2pn0af6kxv3pj232rq,osmo1gjvxngf5f8y399m03z0pcsj72u024vugsfel8l,273948,4.088581e+20,1.140761e+20,evmos1gjvxngf5f8y399m03z0pcsj72u024vug6nmpt9,4.073588e+20,-3.070184e+20
4,5,evmos1mfgtad38nud33el6tcrpfq7smn4j4urykn9xdl,osmo1gjvxngf5f8y399m03z0pcsj72u024vugsfel8l,273948,1.576234e+20,4.397875e+19,evmos1gjvxngf5f8y399m03z0pcsj72u024vug6nmpt9,4.073588e+20,-5.578371e+19


In [26]:
# generate a new results dataframe with the merged events in the same format as the one for claims
merged_output = []
for index, row in merge_event_copy.iterrows():
    merged_output.append((row['id'], row['sender_evmos_prefix'], "ACTION_IBC_TRANSFER", row['height'], row['sender_genesis_claim_record'], row['sender_genesis_claim_record'], row['claimed_coins'], row['lost_amount']))
merged_result = pd.DataFrame(merged_output, columns=['id', 'sender','event_type', 'height', 'initial_claimable_amount', 'claimable_at_event', 'amount_claimed', 'lost_amount'])
merged_result.head()

,id,sender,event_type,height,initial_claimable_amount,claimable_at_event,amount_claimed,lost_amount
0,1,evmos14h3yn7h4lezzmtqwn7usuzmd6snnugpsndzmrz,ACTION_IBC_TRANSFER,273246,1.835300e+20,1.835300e+20,1.462502e+20,-1.003677e+20
1,2,evmos1ermc8wk0yqjwfl47xdcn429hz9zm04v0lngn4f,ACTION_IBC_TRANSFER,273249,3.514016e+20,3.514016e+20,2.835015e+20,-1.956511e+20
2,3,evmos1rckjyht7dl0uhm9qk8k6f2llk0mtz5cygdz2pn,ACTION_IBC_TRANSFER,273353,5.192732e+20,5.192732e+20,3.101760e+20,-1.803577e+20
3,4,evmos1gjvxngf5f8y399m03z0pcsj72u024vug6nmpt9,ACTION_IBC_TRANSFER,273948,4.073588e+20,4.073588e+20,4.088581e+20,-3.070184e+20
4,5,evmos1gjvxngf5f8y399m03z0pcsj72u024vug6nmpt9,ACTION_IBC_TRANSFER,273948,4.073588e+20,4.073588e+20,1.576234e+20,-5.578371e+19


In [27]:
# Address whose decay loss is bigger than 0
impacted_merged_event_results = merged_result.loc[merged_result['lost_amount'] > 0]

In [ ]:
impacted_merged_event_results.sort_values(by=['sender']).to_csv('merge_decay_loss_results.csv', index=False)

### Concatenate Merge and Claim events results

In [28]:
# join `impacted_merged_event_results` with `impacted_claim_events_results`
impacted_events_results = pd.concat([impacted_merged_event_results, impacted_claim_events_results])

In [29]:
# Generate csv file with results for impacted events sorted by address
impacted_events_results.sort_values(by=['sender']).to_csv('decay_loss_results.csv', index=False)

In [30]:
# create a pivot table groupb by sender in order to get the total `lost_amount` by address
impacted_events_results_pivot = impacted_events_results.pivot_table(index=['sender'], values=['lost_amount'], aggfunc=np.sum)
impacted_events_results_pivot.head()

,lost_amount
sender,
evmos1000z4mk9jtr282yxmc3577qr9h6sprlu3tsz6l,1.187456e+19
evmos10078ys6pyrqhr9sdrphdtgth2u3t5kmz644539,2.616281e+18
evmos100a9fpmq7rsjkxyryauhr9tt5z9c0hwgwknct2,2.911406e+20
evmos100cmf99uzq034fpzqeyqgp5aypdhx93m7w0er6,1.648928e+15
evmos100da25cj3dk22u5n9avllrmnhy08ppkkm6wpv7,2.876441e+18


In [31]:
# Generate go file with the total `lost_amount` by address for it to be used on Evmos v12 upgrade
amount_of_accounts = impacted_events_results_pivot.count()
header = "package v12\n\n// Accounts holds the missing claim amount to the corresponding account\nvar Accounts = ["+str(amount_of_accounts[0])+"][2]string{\n"

account_string = ""
for index, row in impacted_events_results_pivot.iterrows():
    lost_amount = "{:.0f}".format(row['lost_amount'])
    account_string = account_string + "\t{\""+str(index)+"\", \""+lost_amount+"\"},\n"
    
file_string = header + account_string + "}"
with open("accounts.go", "w") as text_file:
    text_file.write(file_string)

# Observations

### Affected accounts from claims

- There were a total of `37475` amount of events in which there was a decay issue 

In [32]:
# Total amount of addresses that were affected by the decay bug
impacted_events_results.count()

id                          37475
sender                      37475
event_type                  37475
height                      37475
initial_claimable_amount    37449
claimable_at_event          37475
amount_claimed              37475
lost_amount                 37475
expected_claim              37113
dtype: int64

- There is a total amount of `19164` affected addresses

In [33]:
# Total amount of unique addresses that were affected by the decay bug
impacted_events_results['sender'].nunique()

19164

- Total amount of Evmos tokens lost on claims: `169580.45688720542`

In [34]:
# Total amount of lost tokens on Evmos denomination
impacted_events_results['lost_amount'].sum() / 1000000000000000000

169580.45688720542

- From the affected accounts we can see that the block ranges were in between `265410` - `5073387`

In [35]:
# Min height of impacted events
impacted_events_results.sort_values(by=['height']).head(5)

,id,sender,event_type,height,initial_claimable_amount,claimable_at_event,amount_claimed,lost_amount,expected_claim
235302,235303,evmos1al2j48j9fl4e4k8defvgc75hq0t8ehlhupl36d,ACTION_EVM,265410,3.226298e+19,3.226298e+19,6.387250e+18,1.678495e+18,8.065744e+18
235303,235304,evmos14v4n0umtecu38ca67s04u5xnajnghglk4fs23a,ACTION_DELEGATE,265426,4.277243e+19,4.277243e+19,8.467625e+18,2.225482e+18,1.069311e+19
235304,235305,evmos1545k52p32uskfp3aar7sewd6sc3spe9kgyecnw,ACTION_VOTE,265442,1.473063e+19,1.473063e+19,2.916138e+18,7.665203e+17,3.682658e+18
235305,235306,evmos1t8ct8gyv7qpnmsyk45p72eq4vnemat23fpt5sx,ACTION_DELEGATE,265443,7.050327e+18,7.050327e+18,1.395710e+18,3.668714e+17,1.762582e+18
235306,235307,evmos1ktel9ec72nd6addnfvdv2v52zjyqkvcn6f9wsh,ACTION_EVM,265452,7.050327e+18,7.050327e+18,1.395692e+18,3.668893e+17,1.762582e+18


In [36]:
# Max height of impacted events
impacted_events_results.sort_values(by=['height'], ascending=False).head(5)

,id,sender,event_type,height,initial_claimable_amount,claimable_at_event,amount_claimed,lost_amount,expected_claim
520042,520043,evmos1pu8sjzk8wwhp3272z948uajfdz0kkjf6l955jd,ACTION_EVM,5073387,7.050327e+18,7.050327e+18,5.019794e+14,1.762080e+18,1.762582e+18
520041,520042,evmos1a66kkjer33wyzhpy7dw839ltk78f4ytkp3upm4,ACTION_EVM,5072234,7.050327e+18,7.050327e+18,1.223862e+15,1.761358e+18,1.762582e+18
1021,1022,evmos1pqsf299rc3smtzvd9pcvz6j9hdx4vwwv2v42wu,ACTION_IBC_TRANSFER,5071745,5.192732e+20,5.192732e+20,4.815875e+17,1.293367e+20,NaN
520032,520033,evmos1ama6mf2nn7dw9jruz9gumvs9wwvrsv6z4pcc9h,ACTION_IBC_TRANSFER,5071523,7.645915e+19,7.645915e+19,1.809251e+16,1.909670e+19,1.911479e+19
520026,520027,evmos19vra3y4cg8g7aymfv2gu242h4kcuytl2fufu25,ACTION_IBC_TRANSFER,5071272,4.485653e+19,4.485653e+19,1.161362e+16,1.120252e+19,1.121413e+19


- Within the affected set of accounts, `60` performed double claims

In [37]:
# The following addresses are the ones within the affected groups who performed double claims.
# The decission was to include both claims in the calculation of the lost amount.

# look for events within impacted_events_results that have the same address and the same event_type
event_type_per_address = impacted_events_results.groupby(['sender', 'event_type']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
# get the count of event_type_per_address that have more than 1 event
affected_address_with_double_claim = event_type_per_address.loc[event_type_per_address['counts'] > 1]
affected_address_with_double_claim.count()

sender        60
event_type    60
counts        60
dtype: int64